In [1]:
# Load libraries
library(foreign)
library(lubridate)
library(caret)
library(Metrics)
library(ggplot2)
library(vip)
library(earth)
library(plyr)  
library(openxlsx)


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: lattice

Loading required package: ggplot2


Attaching package: 'Metrics'


The following objects are masked from 'package:caret':

    precision, recall



Attaching package: 'vip'


The following object is masked from 'package:utils':

    vi


Loading required package: Formula

Loading required package: plotmo

Loading required package: plotrix

Loading required package: TeachingDemos



In [2]:
source("1 - Files/methods.r")
source("1 - Files/functions.r")

In [3]:
input_file <- "Input.csv"
size=500
output_file <- "Output.csv"

# Preprocessing

In [23]:
# read network raw data
# read_raw_data()

input_data = read_case_data(size, input_file)

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
"invalid input found on input connection '2 - Data/Lineload.csv'"
Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
"EOF within quoted string"


[1] "The following lines will be excluded because they have more than 400 missing values"
 [1] "Line.293"     "Line.294"     "Line.304.508" "Line.499"     "Line.500"    
 [6] "Line.502"     "Line.503"     "Line.507.306" "Line.517"     "Line.518"    
[11] "Line.H"       "Line.460"     "Line.479"     "Line.515"     "Line.596"    
[16] "Line.906"     "Line.317.."   "Line.331.."   "Line.332.."   "Line.449.1"  
[21] "Line.450.2"   "Line.491.1"   "Line.492.2"   "Line.501.."   "Line.521.."  
[26] "Line.952"     "Line.NOR.HAN" "Line.304"     "Line.306"     "Line.444"    
[31] "Line.476"     "Line.477"     "Line.293.294" "Line.413.436" "Line.501"    
[36] "Line.533"     "Line.561"     "Line.12.514"  "Line.321"     "Line.510"    
[41] "Line.512.514" "Line.515.516" "Line.572.573" "Line.NOAU.WS" "Line.NODO.BL"
[46] "Line.NOHA.BL" "Line.NOHA.RT" "Line.420.424" "Line.487"     "Line.488"    


In [24]:
covariates = input_data$covariates
covariates[,-1] = sapply(input_data$covariates[,-1] ,as.numeric)

lines = input_data$lines
lines[,-1] = sapply(input_data$lines[,-1] ,as.numeric)

t_lines = input_data$t_lines

methods = input_data$methods

# Training and estimation

In [26]:
set.seed(123)  
results = data.frame(row.names = t_lines)

for (line in c(t_lines)){ # 
    
    #add lagged line if not included in covariates and delete NAs
    covariates_l <- covariates
    covariates_l <- merge(covariates_l, lines[,c("Time",line)] ,by="Time")
    if (!(paste(line, ".lag", sep="") %in% names(covariates_l))){
        covariates_l <- merge(covariates_l, lagged(lines[,c("Time",line)]), by="Time")
    }
    covariates_l <- na.omit(covariates_l)
    
    # select training and test data
    trainingRowIndex <- sample(1:nrow(covariates_l), 0.7*nrow(covariates_l))  
    trainingData <- covariates_l[sample(trainingRowIndex, min(size, nrow(trainingRowIndex))), ]  
    testData  <- covariates_l[-trainingRowIndex, ] 
    
    # perform training and estimation
    for (method in methods){
        if (method == "MARS") {
            results[line, method] <- mars(line, trainingData, testData)
            
        } else if (method == "Linear regression") {
            results[line, method] <- lin_reg(line, trainingData, testData)
        
        } else if (method == "Random forest") {
            results[line, method] <- ran_forest(line, trainingData, testData)
            
        } else if (method == "Multiple linear regression") {
            print('Function is not implemented yet')
            
        } else if (method == "kNN") {
            results[line, method] <- kNN(line, trainingData, testData)
            
        } else if (method == "Support vector regression") {
            results[line, method] <- SVR(line, trainingData, testData)   
        }
    }
}

Warning message in preProcess.default(method = c("center", "scale"), x = structure(c(1454439600, :
"These variables have zero variances: Line.313.lag, Line.314.lag"
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: Line.313.lag, Line.314.lag"
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: Line.313.lag, Line.314.lag"
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: Line.313.lag, Line.314.lag"
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: Line.313.lag, Line.314.lag"
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: Line.313.lag, Line.314.lag"
Warning message in preProcess.default(thresh 

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: Line.313.lag, Line.314.lag"
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: Line.313.lag, Line.314.lag"


# Return results

In [158]:
input <- read.csv(input_file, fileEncoding="UTF-8-BOM")

write.xlsx(
      input,
      input_file,
      sheetName = 'Input',
      col.names = TRUE,
      row.names = TRUE,
      append = TRUE,
    )

for (method in methods){
    write.xlsx(
      results[method],
      output_file,
      sheetName = String(method),
      col.names = TRUE,
      row.names = TRUE,
      append = TRUE,
    )
}

ERROR: Error in write.xlsx(input, input_file, sheetName = "Input", col.names = TRUE, : could not find function "write.xlsx"
